# Corporate Messaging Case Study

In this Notebook we will create a text message classification process. This corporate message data is from one of the free datasets provided on the [Figure Eight Platform](https://www.figure-eight.com/data-for-everyone/), licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

I use NLP to process text data and then apply a classifier that predicts under which of the three categories each message falls under.

For the NLP part I use a custom transormer, which is in the `custom_transformer.py` file that I import a few lines below.

The whole process is incorporated in a pipeline, which makes very clear the order in which each step is applied and also easier to optimize any step from the data processing to modeling.

For the above-mentioned optimization I use `GridSearchCV` and grid search some parameters in the data transformation steps as well as those for the classifier. After the grid search step we have the optimal parameters for all the steps applied in the pipeline.

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.kontonis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\v.kontonis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\v.kontonis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

## Import a custom transformer
Import custom transformer that extracts whether each text starts with a verb.

In [6]:
from custom_transformer import *

### Create the `load_data` function
This function loads the data from the csv file and creates the X and y

In [4]:
def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y

### Build a pipeline for the whole process, from text data manipulation to classifier
Before modifying the build_model method to include grid search, view the parameters in your pipeline here.

In [7]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

In [8]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

### Create the `build_model` function
This method applies the pipeline and also tries to grid search some parameters in the data transformation steps as well as those for the classifier! Browse the parameters you can search above.
It returns a GridSearchCV object.

In [11]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

    return cv

### Create `save_final_model` function
This function saves the best model from the `build_model` step.

In [17]:
def save_final_model(model, filename='final_model.pkl'):
    joblib.dump(model, filename)

### Run program to test
Running grid search can take a while, especially if you are searching over a lot of parameters! If you want to reduce it to a few minutes, try commenting out some of your parameters to grid search over just 1 or 2 parameters with a small number of values each.

In [12]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)


def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    
    save_final_model(model, filename='final_model.pkl')

main()

C:\Users\v.kontonis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 972 candidates, totalling 2916 fits
[CV] clf__min_samples_split=2, clf__n_estimators=50, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 
[CV]  clf__min_samples_split=2, clf__n_estimators=50, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.947, total=   5.4s
[CV] clf__min_samples_split=2, clf__n_estimators=50, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s


[CV]  clf__min_samples_split=2, clf__n_estimators=50, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.928, total=   5.6s
[CV] clf__min_samples_split=2, clf__n_estimators=50, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.8s remaining:    0.0s


...

[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 0.8, 'starting_verb': 1}, score=0.952, total=   5.8s
[CV] clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 
[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, score=0.935, t

[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 0.5, 'starting_verb': 1}, score=0.937, total=   5.6s
[CV] clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 0.5, 'starting_verb': 1} 
[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 1), features__transformer_weights={'text_pipeline': 0.5, 'starting_verb': 1}, score=0.940

[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 0.8, 'starting_verb': 1}, score=0.927, total=   5.7s
[CV] clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 0.8, 'starting_verb': 1} 
[CV]  clf__min_samples_split=4, clf__n_estimators=200, features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__max_df=1.0, features__text_pipeline__vect__max_features=10000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 0.8, 'starting_verb': 1}, score=0.950

[Parallel(n_jobs=1)]: Done 2916 out of 2916 | elapsed: 270.0min finished


Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[100   0  28]
 [  1  25   3]
 [  4   1 439]]
Accuracy: 0.9384359400998337

Best Parameters: {'clf__min_samples_split': 4, 'clf__n_estimators': 50, 'features__text_pipeline__tfidf__use_idf': True, 'features__text_pipeline__vect__max_df': 0.5, 'features__text_pipeline__vect__max_features': 5000, 'features__text_pipeline__vect__ngram_range': (1, 2), 'features__transformer_weights': {'text_pipeline': 0.8, 'starting_verb': 1}}


### Import the final and saved model to make predictions later on
In the cells below we import the saved model as a nest step, in order to use it for new messages prediction.

In [ ]:
def load_final_model(filename='final_model.pkl'):
    return joblib.load(filename)

In [ ]:
model = load_final_model()